In [138]:
import openreview
import gensim
import nltk
from collections import Counter, defaultdict as dd
import numpy as np
from pivottablejs import pivot_ui
import pickle
import pandas as pd
import qgrid
from pprint import pprint
import re 
import jellyfish
import time

from plotly.offline import init_notebook_mode
import plotly.graph_objs as go
init_notebook_mode()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [ ]:
c = openreview.Client(baseurl='https://openreview.net')

In [ ]:
notes = c.get_notes(invitation='ICLR.cc/2019/Conference/-/Blind_Submission') + \
    c.get_notes(invitation='ICLR.cc/2019/Conference/-/Blind_Submission', offset=1000, limit=1000)

In [ ]:
note = notes[0]
jsnote = note.to_json()
jsnote

In [ ]:
comments = c.get_notes(forum="H1lqZhRcFm")
refs = c.get_references(referent="H1lqZhRcFm")
refs[0]

In [ ]:
from collections import defaultdict as dd
import re 
def ratings_and_confidence(notes):
    ratings = []
    confs = []
    for note in notes:
        content = note.to_json()['content']
        paper = note.to_json()['forum']
        if 'rating' in content:
            score = int(re.findall(pattern='\d+', string=content['rating'])[0])
            conf = int(re.findall(pattern='\d+', string=content['confidence'])[0])
            ratings.append(score)
            confs.append(conf)
    return ratings, confs

In [ ]:
paper_notes = c.get_notes(invitation='ICLR.cc/2019/Conference/-/Blind_Submission') + \
    c.get_notes(invitation='ICLR.cc/2019/Conference/-/Blind_Submission', offset=1000, limit=1000)

In [116]:
paper_notes2 = c.get_notes(invitation='ICLR.cc/2019/Conference/-/Blind_Submission') + \
    c.get_notes(invitation='ICLR.cc/2019/Conference/-/Blind_Submission', offset=1000, limit=1000)

In [ ]:
from pprint import pprint 

for ix in range(len(paper_notes)):
    print(ix)
    forumId = paper_notes[ix].forum
    forum_notes = c.get_notes(forum=forumId)
    ratings, confidence = ratings_and_confidence(forum_notes)
    paper_notes[ix].ratings = ratings
    paper_notes[ix].confidence = confidence
#     if ix > 10:
#         break

with open('papers2019_icml.pcl', 'wb+') as f:
    pickle.dump(paper_notes, f)

In [14]:
with open('papers2019_icml.pcl', 'rb+') as f:
    paper_notes = pickle.load(f)

In [ ]:
def number_of_ratings(paper_notes):
    num_ratings = []
    for paper in paper_notes:
        num_ratings.append(len(paper.ratings))
    return Counter(num_ratings)

In [ ]:
number_of_ratings(paper_notes)

In [ ]:
def average_rating(paper_notes, field='ratings'):
    ratings = []
    for paper in paper_notes:
        ratings.extend(paper.__getattribute__(field))
    return np.mean(ratings), Counter(ratings)

In [ ]:
average_rating(paper_notes)

In [ ]:
average_rating(paper_notes, 'confidence')

In [ ]:
from scipy.stats import pearsonr
ratings = []
confidence = []
for paper in paper_notes:
    ratings.extend(paper.__getattribute__('ratings'))
    confidence.extend(paper.__getattribute__('confidence'))
    
# pearsonr(ratings, confidence)
len(ratings), len(confidence)

In [ ]:
copy = paper_notes.copy()

In [ ]:
p = paper_notes[0]
getattr(p, 'confidence', '')
print(p)

In [97]:
rows = []
for paper in paper_notes:
    row = dict(tldr = paper.content.get('TL;DR', ''),
              ratings = getattr(paper, 'ratings', []),
              confidence = getattr(paper, 'confidence', []),
              title = paper.content['title'],
              avg_rating = np.mean(getattr(paper, 'ratings', [])),
              avg_confidence = np.mean(getattr(paper, 'confidence', [])))
    rows.append(row)
df = pd.DataFrame(rows)
df.index.name = 'Index'
df = df.iloc[:, [4,1,0,5,3,2]]

In [98]:
df.head()

,title,avg_rating,avg_confidence,tldr,ratings,confidence
Index,,,,,,
0,State-Regularized Recurrent Networks,5.666667,4.666667,"We introduce stochastic state transition mechanism to RNNs, simplifies finite state automata (DFA) extraction, forces RNNs to operate more like automata with external memory, better extrapolation behavior and interpretability.","[6, 6, 5]","[4, 5, 5]"
1,Deep Denoising: Rate-Optimal Recovery of Structured Signals with a Deep Prior,5.666667,3.333333,"By analyzing an algorithms minimizing a non-convex loss, we show that all but a small fraction of noise can be removed from an image using a deep neural network based generative prior.","[6, 5, 6]","[3, 3, 4]"
2,The Deep Weight Prior,6.333333,3.666667,An empirical prior for convolutional layers in Bayesian neural networks that improves learning on small datasets.,"[4, 8, 7]","[4, 4, 3]"
3,CoT: Cooperative Training for Generative Modeling of Discrete Data,7.000000,2.666667,"We proposed Cooperative Training, a novel training algorithm for generative modeling of discrete data.","[7, 7, 7]","[2, 2, 4]"
4,Adversarial Information Factorization,6.000000,4.000000,Learn representations for images that factor out a single attribute.,"[6, 6, 6]","[4, 4, 4]"


In [ ]:
pivot_ui(df)

In [24]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
pd.set_option('display.max_colwidth', -1)

In [26]:
df.head()

,confidence,ratings,title,tldr
0,"[4, 5, 5]","[6, 6, 5]",State-Regularized Recurrent Networks,"We introduce stochastic state transition mechanism to RNNs, simplifies finite state automata (DFA) extraction, forces RNNs to operate more like automata with external memory, better extrapolation behavior and interpretability."
1,"[3, 3, 4]","[6, 5, 6]",Deep Denoising: Rate-Optimal Recovery of Structured Signals with a Deep Prior,"By analyzing an algorithms minimizing a non-convex loss, we show that all but a small fraction of noise can be removed from an image using a deep neural network based generative prior."
2,"[4, 4, 3]","[4, 8, 7]",The Deep Weight Prior,An empirical prior for convolutional layers in Bayesian neural networks that improves learning on small datasets.
3,"[2, 2, 4]","[7, 7, 7]",CoT: Cooperative Training for Generative Modeling of Discrete Data,"We proposed Cooperative Training, a novel training algorithm for generative modeling of discrete data."
4,"[4, 4, 4]","[6, 6, 6]",Adversarial Information Factorization,Learn representations for images that factor out a single attribute.


In [50]:
grid_options = {
            'fullWidthRows': True,
            'syncColumnCellResize': True,
            'forceFitColumns': True,
            'defaultColumnWidth': 150,
            'rowHeight': 28,
            'enableColumnReorder': False,
            'enableTextSelectionOnCells': True,
            'editable': True,
            'autoEdit': False,
            'explicitInitialization': True,
            'maxVisibleRows': 15,
            'minVisibleRows': 8
        }
# grid_options = {
#     'rowHeight': 50
# }

In [61]:
row = dict(title= 'CoT: Cooperative Training for Generative Modeling of Discrete Data',
tldr= 'We proposed Cooperative Training, a novel training algorithm for generative modeling of discrete data. We introduce stochastic state transition mechanism to RNNs, simplifies finite state automata (DFA) extraction, forces RNNs to operate more like automata with external memory, better extrapolation behavior and interpretability.')
df = pd.DataFrame([row])

In [82]:
pd.set_option('display.max_colwidth', -1)
df.head()

,avg_confidence,avg_rating,confidence,ratings,title,tldr
Index,,,,,,
0,4.666667,5.666667,"[4, 5, 5]","[6, 6, 5]",State-Regularized Recurrent Networks,"We introduce stochastic state transition mechanism to RNNs, simplifies finite state automata (DFA) extraction, forces RNNs to operate more like automata with external memory, better extrapolation behavior and interpretability."
1,3.333333,5.666667,"[3, 3, 4]","[6, 5, 6]",Deep Denoising: Rate-Optimal Recovery of Structured Signals with a Deep Prior,"By analyzing an algorithms minimizing a non-convex loss, we show that all but a small fraction of noise can be removed from an image using a deep neural network based generative prior."
2,3.666667,6.333333,"[4, 4, 3]","[4, 8, 7]",The Deep Weight Prior,An empirical prior for convolutional layers in Bayesian neural networks that improves learning on small datasets.
3,2.666667,7.000000,"[2, 2, 4]","[7, 7, 7]",CoT: Cooperative Training for Generative Modeling of Discrete Data,"We proposed Cooperative Training, a novel training algorithm for generative modeling of discrete data."
4,4.000000,6.000000,"[4, 4, 4]","[6, 6, 6]",Adversarial Information Factorization,Learn representations for images that factor out a single attribute.


In [99]:
# col_opts = { 
#     'editable': False,
#     'toolTip': "Not editable"
# }

col_defs = {
    name: {
        'width': 50
    } for name in ['Index', 'avg_confidence', 'avg_rating', 'confidence', 'ratings']
}
qgrid.show_grid(df, column_definitions=col_defs)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [ ]:
pprint(paper_notes[0].to_json())

In [109]:
client = openreview.Client(baseurl='https://openreview.net')
# notes = c.get_notes(limit = 10000)

In [114]:
review_iterator = openreview.tools.iterget_notes(client, invitation='ICLR.cc/2019/Conference/-/Paper.*/Official_Review')
   

{'cdate': 1542286428411,
 'content': {'confidence': '4: The reviewer is confident but not absolutely '
                           'certain that the evaluation is correct',
             'rating': '5: Marginally below acceptance threshold',
             'review': 'Summary. The authors propose a novel adversarial '
                       'training method, e2SAD, that relies on a two-step '
                       'process for generating sets of two training '
                       'adversarial samples for each clean training sample. '
                       'The first step is a classical FGSM that yields the '
                       'first adversarial sample. The second adversarial '
                       'sample is calculated with a FGSM that is based on the '
                       'cross-entropy between the probabilities generated by '
                       'the first adversarial sample and the probabilities '
                       'generated by the second adversarial sample. The me

In [117]:
paper_notes[0]

Note(id = 'HJf7ts0cFm',original = 'B1l3kE55KQ',number = 431,cdate = 1538087803027,tcdate = 1538087803027,tmdate = 1542208692666,ddate = None,content = {'title': 'State-Regularized Recurrent Networks', 'abstract': "Recurrent networks are a widely used class of neural architectures.  They have, however, two shortcomings. First, it is difficult to understand what exactly they learn. Second, they tend to work poorly on sequences requiring long-term memorization, despite having this capacity in principle. We aim to address both shortcomings with a class of recurrent networks that use a stochastic state transition mechanism between cell applications. This mechanism, which we term state-regularization, makes RNNs transition between a finite set of learnable states. We show that state-regularization (a) simplifies the extraction of finite state automata modeling an RNN's state transition dynamics, and (b) forces RNNs to operate more like automata with external memory and less like finite state

In [5]:
def ratings_and_confidence():
    client = openreview.Client(baseurl='https://openreview.net')
    review_iterator = openreview.tools.iterget_notes(client, invitation='ICLR.cc/2019/Conference/-/Paper.*/Official_Review')
    ratings = dd(list)
    confidence = dd(list)
    for review in review_iterator:
        ratings[review.forum].append(int(re.findall(pattern='\d+', string=review.content['rating'])[0]))
        confidence[review.forum].append(int(re.findall(pattern='\d+', string=review.content['confidence'])[0]))
    return ratings, confidence

In [6]:
def all_papers():
    client = openreview.Client(baseurl='https://openreview.net')
    paper_iterator = openreview.tools.iterget_notes(client, invitation='ICLR.cc/2019/Conference/-/Blind_Submission')
    return list(paper_iterator)

In [7]:
ratings, confidence = ratings_and_confidence()
papers = all_papers()

In [8]:
def add_ratings(papers, ratings, confidence):
    for ix in range(len(papers)):
        forum = papers[ix].forum
        papers[ix].ratings = ratings.get(forum, [])
        papers[ix].confidence = confidence.get(forum, [])

In [9]:
add_ratings(papers, ratings, confidence)

In [192]:
def get_keywords(papers):
    return list(set([word.strip().lower() for p in papers for word in p.content['keywords']]))

def get_similarity(keywords):
    sim = dict()
    N = len(keywords)
    for i in range(N-1):
        for j in range(i+1, N):
            topic1 = keywords[i]
            topic2 = keywords[j]
            dist = len(set(topic1).intersection(set(topic2)))/len(set(max([topic1, topic2], key=lambda v: len(v))))
            if topic1 != topic2 and dist > 0.75:
                dist = jellyfish.levenshtein_distance(topic1, topic2)
                sim[topic1, topic2] = dist
    return sim

def get_top_matches(sim):
    matches = dd(list)
    for pair in sim:
        if sim[pair] == 1:
            matches[pair[0]].append(pair[1])
            matches[pair[1]].append(pair[0])
    return matches    

In [193]:
keywords = get_keywords(papers)
start = time.time()
sim = get_similarity(keywords) # this takes about 2 minutes 
finish = time.time()
print('Similarity computed in {:.2f} sec'.format(finish - start))

Similarity computed in 118.28 sec


In [201]:
keys = []
for key in sim:
    if 'active learning' in key:
        keys.append(key)
pprint(sorted([(key, sim[key]) for key in keys], key=lambda v: v[1]))

[(('active learning', 'active  learning'), 1),
 (('active learning', 'machine learning'), 3),
 (('active tracking', 'active learning'), 4),
 (('active learning', 'code learning'), 4),
 (('active learning', 'inductive learning'), 4),
 (('feature learning', 'active learning'), 5),
 (('active learning', 'td learning'), 5),
 (('active learning', 'predictive learning'), 5),
 (('active learning', 'set learning'), 5),
 (('active learning', 'meta learning'), 5),
 (('active learning', 'metric learning'), 5),
 (('hebbian learning', 'active learning'), 6),
 (('active learning', 'robust learning'), 6),
 (('active learning', 'concept learning'), 6),
 (('active learning', 'structure learning'), 6),
 (('active learning', 'spectral learning'), 6),
 (('active learning', 'tactile sensing'), 6),
 (('active learning', 'relation learning'), 6),
 (('active learning', 'generative learning'), 6),
 (('active learning', 'stream learning'), 6),
 (('active learning', 'distance learning'), 6),
 (('active learning'

In [202]:
matches = get_top_matches(sim)

In [203]:
matches

defaultdict(list,
            {'3d learning': ['td learning'],
             'activation function': ['activation functions'],
             'activation functions': ['activation function'],
             'active  learning': ['active learning'],
             'active learning': ['active  learning'],
             'adam': ['admm'],
             'adaptive method': ['adaptive methods'],
             'adaptive methods': ['adaptive method'],
             'admm': ['adam'],
             'adversarial attack': ['adversarial attacks'],
             'adversarial attacks': ['adversarial attack'],
             'adversarial defence': ['adversarial defense'],
             'adversarial defense': ['adversarial defence'],
             'adversarial example': ['adversarial examples'],
             'adversarial examples': ['adversarial example'],
             'adversarial perturbation': ['adversarial perturbations'],
             'adversarial perturbations': ['adversarial perturbation'],
             'algorithm':

In [204]:
def get_clusters(matches):
    clusters = dict()
    for topic in matches:
        values = matches[topic]
        clusters[topic] = topic
        for v in values:
            if len(matches.get(v, [])) > len(values) or \
            (len(matches.get(v, [])) == len(values) and topic > v):
                clusters[topic] = v
    return clusters

In [206]:
clusters = get_clusters(matches)

In [208]:
def get_df(papers, clusters):
    rows = []
    for paper in papers:
        topic = paper.content['keywords'][0] if len(paper.content['keywords']) else ''
        cluster = clusters.get(topic.lower(), topic.lower() if topic else None)
        row = dict(tldr = paper.content.get('TL;DR', ''),
                  ratings = getattr(paper, 'ratings', []),
                  confidence = getattr(paper, 'confidence', []),
                  title = paper.content['title'],
                  avg_rating = np.mean(getattr(paper, 'ratings', [])),
                  avg_confidence = np.mean(getattr(paper, 'confidence', [])),
                  topic = cluster,
                  url = "https://openreview.net/forum?id=" + paper.forum
                  )
        rows.append(row)
    df = pd.DataFrame(rows)
    df.index.name = 'Index'
    df = df.iloc[:, [4,1,0,6,5,3,2,7]]
    return df 

In [209]:
df = get_df(papers, clusters)

In [210]:
df.head()

,title,avg_rating,avg_confidence,topic,tldr,ratings,confidence,url
Index,,,,,,,,
0,G-SGD: Optimizing ReLU Neural Networks in its ...,6.666667,3.000000,optimisation,,"[7, 7, 6]","[2, 3, 4]",https://openreview.net/forum?id=SyxfEn09Y7
1,ADef: an Iterative Algorithm to Construct Adve...,6.000000,3.333333,adversarial example,"We propose a new, efficient algorithm to const...","[7, 7, 4]","[4, 3, 3]",https://openreview.net/forum?id=Hk4dFjR5K7
2,Generative Code Modeling with Graphs,6.333333,4.333333,generative models,Representing programs as graphs including sema...,"[7, 7, 5]","[4, 4, 5]",https://openreview.net/forum?id=Bke4KsA5FX
3,Quality Evaluation of GANs Using Cross Local I...,5.000000,4.000000,generative adversarial network,We propose a new metric for evaluating GAN mod...,"[4, 6, 5]","[3, 4, 5]",https://openreview.net/forum?id=BJgYl205tQ
4,L-Shapley and C-Shapley: Efficient Model Inter...,6.333333,3.000000,model interpretation,We develop two linear-complexity algorithms fo...,"[7, 7, 5]","[3, 2, 4]",https://openreview.net/forum?id=S1E3Ko09F7


In [213]:
col_defs = {
    name: {
        'width': 50
    } for name in ['Index', 'confidence', 'ratings']
}
col_defs['title'] = {'width': 500}
col_defs['avg_rating'] = {'width': 100}
col_defs['tldr'] = {'width': 1000}
col_defs['url'] = {'width': 300}
col_defs['topic'] = {'width': 200}
grid_options={'forceFitColumns': False, 'defaultColumnWidth': 100}
qgrid.show_grid(df, column_definitions=col_defs, grid_options = grid_options, precision=1,)

TypeError: show_grid() got an unexpected keyword argument 'export_mode'

In [214]:
?qgrid.show_grid

In [157]:
for topic in top:
    if len(top[topic]) > 1:
        print(topic, top[topic])

non-convex optimization ['non convex optimization', 'nonconvex optimization']
multi-agent reinforcement learning ['multiagent reinforcement learning', 'multi agent reinforcement learning']
variational inference ['variational inference.', 'variational lnference']
multiagent reinforcement learning ['multi-agent reinforcement learning', 'multi agent reinforcement learning']
multiagent ['multi agent', 'multi-agent']
neural networks ['neural network', 'neural-networks']
autoencoders ['autoencoder', 'auto-encoders']
over-parameterization ['overparameterization', 'over-parametrization']
variational auto encoder ['variational autoencoder', 'variational auto encoders', 'variational auto-encoder']
generative adversarial networks ['generative adversarial network', 'generative adversarial networks.']
generative models ['generatice models', 'generative model']
variational autoencoder ['variational auto encoder', 'variational auto-encoder', 'variational autoencoders']
deep-learning ['deep learning',

In [156]:
top['non convex optimization']

['non-convex optimization', 'nonconvex optimization']

In [142]:
l = []
for pair in sim:
    if sim[pair] == 1:
        l.append(sorted(pair))
sorted(l)
        

[['3d learning', 'td learning'],
 ['3d learning', 'td learning'],
 ['activation function', 'activation functions'],
 ['activation function', 'activation functions'],
 ['active  learning', 'active learning'],
 ['active  learning', 'active learning'],
 ['adam', 'admm'],
 ['adam', 'admm'],
 ['adaptive method', 'adaptive methods'],
 ['adaptive method', 'adaptive methods'],
 ['adversarial attack', 'adversarial attacks'],
 ['adversarial attack', 'adversarial attacks'],
 ['adversarial defence', 'adversarial defense'],
 ['adversarial defence', 'adversarial defense'],
 ['adversarial example', 'adversarial examples'],
 ['adversarial example', 'adversarial examples'],
 ['adversarial perturbation', 'adversarial perturbations'],
 ['adversarial perturbation', 'adversarial perturbations'],
 ['algorithm', 'algorithms'],
 ['algorithm', 'algorithms'],
 ['amortised inference', 'amortized inference'],
 ['amortised inference', 'amortized inference'],
 ['artificial neural network', 'artificial neural networ

In [92]:
ix = 0 
for item in sim.items():
    print(item)
    ix += 1
    if ix > 10:
        break

(("nesterov's method", "liapunov's method"), 6)
(("nesterov's method", 'variational models'), 15)
(("nesterov's method", 'smoothed gradient'), 16)
(("nesterov's method", 'generative models'), 13)
(("nesterov's method", 'recommender systems'), 16)
(("nesterov's method", 'adversarial methods'), 11)
(("nesterov's method", 'kernel methods'), 9)
(('state equation', 'data selection'), 7)
(('state equation', 'set reconstruction'), 10)
(('state equation', 'regression tree'), 13)
(('state equation', 'sequence to sequence'), 15)


In [23]:
pprint(papers[0].to_json())

{'cdate': 1538087978124,
 'content': {'_bibtex': '@inproceedings{    \n'
                        'anonymous2019g-sgd:,    \n'
                        'title={G-SGD: Optimizing ReLU Neural Networks in its '
                        'Positively Scale-Invariant Space},    \n'
                        'author={Anonymous},    \n'
                        'booktitle={Submitted to International Conference on '
                        'Learning Representations},    \n'
                        'year={2019},    \n'
                        'url={https://openreview.net/forum?id=SyxfEn09Y7},    \n'
                        'note={under review}    \n'
                        '}',
             'abstract': 'It is well known that neural networks with rectified '
                         'linear units (ReLU) activation functions are '
                         'positively scale-invariant. Conventional algorithms '
                         'like stochastic gradient descent optimize the neural '
              

In [19]:
qgrid.show_grid?

In [138]:
ratings, confidence = ratings_and_confidence()

In [139]:
len(ratings), len(confidence)

(1565, 1565)

In [142]:
list(ratings.items())[0]

('BklpOo09tQ', ['5', '6', '7'])

In [ ]:
N = c.get_notes(invitation="ICLR.cc/2019/Conference/-/Paper.*/Public_Comment")

In [ ]:
paper=  c.get_notes(forum="SyVU6s05K7")
ratings_and_confidence(paper)

In [ ]:
for n in notes:
    print(n.to_json())

In [ ]:
ratings = add_ratings(notes)

In [ ]:
print(c.get_notes(id='SyVU6s05K7')[0])

In [ ]:
from __future__ import print_function
print(c.get_notes(id='r1g2QHW0h7')[0])

In [42]:
with open('cro.features') as f:
    text = f.read()

In [45]:
rpl = []
for split in text.split('|'):
    if '101536' in split :
        rpl.append(split.replace('101536', '-104356'))

In [49]:
'|'+ '|'.join(rpl[:-1]) + '|'

'|1&-104356|100070&-104356|100128&-104356|100132&-104356|100145&-104356|100146&-104356|100164&-104356|100258&-104356|100259&-104356|100260&-104356|100282&-104356|100780&-104356|100783&-104356|100928&-104356|100929&-104356|100930&-104356|100931&-104356|100932&-104356|100933&-104356|100934&-104356|100935&-104356|100936&-104356|100937&-104356|100938&-104356|100939&-104356|100975&-104356|101131&-104356|101399&-104356|17&-104356|-104356&23|-104356&31|-104356&34|-104356&415|-104356&417|-104356&418|-104356&491|-104356&523|-104356&529|-104356&530|-104356&598|-104356&86|'